In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU
from tensorflow.keras.optimizers import Adam

In [3]:
df = pd.read_csv("dst_2014-2023.csv")
df

,Date,Dst
0,2014-01-01 00:00:00,4
1,2014-01-01 01:00:00,3
2,2014-01-01 02:00:00,1
3,2014-01-01 03:00:00,1
4,2014-01-01 04:00:00,-1
...,...,...
87643,2023-12-31 19:00:00,0
87644,2023-12-31 20:00:00,1
87645,2023-12-31 21:00:00,2
87646,2023-12-31 22:00:00,2


In [4]:
df.set_index('Date', inplace=True)
df.isna().sum()

Dst    0
dtype: int64

In [5]:
df_train_valid = df[:int(0.85 * len(df))]

In [6]:
# [[[1], [2], [3], [4], [5]]] [6]
# [[[2], [3], [4], [5], [6]]] [7]
# [[[3], [4], [5], [6], [7]]] [8]

def df_to_X_y(df, window_size=5):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [[a] for a in df_as_np[i:i+window_size]]
    X.append(row)
    label = df_as_np[i+window_size]
    y.append(label)
  return np.array(X), np.array(y)

In [10]:
WINDOW_SIZE = 6
X, y = df_to_X_y(df_train_valid, WINDOW_SIZE)
X.shape, y.shape

((74494, 6, 1, 1), (74494, 1))

In [8]:
def create_model():
    model = Sequential([
        LSTM(64, input_shape=(WINDOW_SIZE, 1), return_sequences=True),
        LSTM(64, return_sequences=False),
        Dense(64, activation ='relu'),
        Dense(1, activation = 'linear')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='huber', metrics=['mae', 'mse'])
    return model

In [11]:
# Menggunakan TimeSeriesSplit untuk cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Menyimpan hasil cross-validation
cv_results = []

for train_index, val_index in tscv.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Inisialisasi scaler
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    # Scaling data
    X_train_scaled = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler_X.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    y_train_scaled = scaler_y.fit_transform(y_train)
    y_val_scaled = scaler_y.transform(y_val)

    # Membuat dan melatih model
    model = create_model()
    model.fit(X_train_scaled, y_train_scaled, epochs=100, batch_size=32, validation_data=(X_val_scaled, y_val_scaled), verbose=1)

    # Evaluasi model
    y_val_pred_scaled = model.predict(X_val_scaled)
    y_val_pred = scaler_y.inverse_transform(y_val_pred_scaled)

    # Menghitung RMSE
    rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    mae = mean_absolute_error(y_val, y_val_pred)
    
    cv_results.append((rmse, mae))

Epoch 1/100
389/389 [==============================] - 14s 23ms/step - loss: 0.1331 - mae: 0.3766 - mse: 0.3330 - val_loss: 0.1025 - val_mae: 0.3188 - val_mse: 0.2455
Epoch 2/100
389/389 [==============================] - 7s 18ms/step - loss: 0.0788 - mae: 0.2842 - mse: 0.1858 - val_loss: 0.0860 - val_mae: 0.2872 - val_mse: 0.1991
Epoch 3/100
389/389 [==============================] - 7s 18ms/step - loss: 0.0623 - mae: 0.2505 - mse: 0.1409 - val_loss: 0.0638 - val_mae: 0.2463 - val_mse: 0.1401
Epoch 4/100
389/389 [==============================] - 8s 19ms/step - loss: 0.0492 - mae: 0.2218 - mse: 0.1081 - val_loss: 0.0528 - val_mae: 0.2239 - val_mse: 0.1139
Epoch 5/100
389/389 [==============================] - 7s 19ms/step - loss: 0.0431 - mae: 0.2072 - mse: 0.0932 - val_loss: 0.0467 - val_mae: 0.2112 - val_mse: 0.0999
Epoch 6/100
389/389 [==============================] - 7s 18ms/step - loss: 0.0382 - mae: 0.1945 - mse: 0.0820 - val_loss: 0.0413 - val_mae: 0.1960 - val_mse: 0.0878
Epo

In [13]:
# Rata-rata dan standar deviasi hasil cross-validation
rmse_scores = [result[0] for result in cv_results]
mae_scores = [result[1] for result in cv_results]

avg_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)
avg_mae = np.mean(mae_scores)
std_mae = np.std(mae_scores)


print(f'Average RMSE: {avg_rmse}, Standard Deviation RMSE: {std_rmse}')
print(f'Average MAE: {avg_mae}, Standard Deviation MAE: {std_mae}')

Average RMSE: 3.26086270616384, Standard Deviation RMSE: 0.4954362861847178
Average MAE: 2.251209139268981, Standard Deviation MAE: 0.2805836788043266
